In [1]:
!pip install allensdk

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache
import allensdk

c:\myprograms\Anaconda\envs\behavis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Confirming your allensdk version
print(f"Your allensdk version is: {allensdk.__version__}")

Your allensdk version is: 2.16.2


In [3]:
# Update this to a valid directory in your filesystem. This is where the data will be stored.
output_dir = "D:\Visual Behavior Neuropixels AI"
DOWNLOAD_COMPLETE_DATASET = False

In [4]:
output_dir = Path(output_dir)
cache = VisualBehaviorNeuropixelsProjectCache.from_s3_cache(cache_dir=output_dir)
# Load latest manifest
cache.load_latest_manifest()

In [5]:
ecephys_sessions = cache.get_ecephys_session_table()
print(f"Total number of ecephys sessions: {len(ecephys_sessions)}")
behavior_sessions = cache.get_behavior_session_table()
print(f"Total number of behavior sessions: {len(behavior_sessions)}")
units = cache.get_unit_table()
print(f"Total number of units: {len(units)}")
probes = cache.get_probe_table()
print(f"Total number of probes: {len(probes)}")
channels = cache.get_channel_table()
print(f"Total number of channels: {len(channels)}")

Total number of ecephys sessions: 103
Total number of behavior sessions: 3424
Total number of units: 319013
Total number of probes: 905
Total number of channels: 347520


In [ ]:
# behavior_sessions['session_type']
mask = (behavior_sessions['session_type'] == 'EPHYS_1_images_G_3uL_reward') | (behavior_sessions['session_type'] == 'EPHYS_1_images_H_3uL_reward')
ephys_behavior_sessions = behavior_sessions[mask]
ecephys_sessions_ids = ephys_behavior_sessions['ecephys_session_id'].values

In [8]:
# behavior_sessions.columns

In [7]:
# behavior_sessions['session_type'].value_counts()

In [9]:
# # mask = behavior_sessions['mouse_id'] == 544835
# mask = behavior_sessions['session_type'] == 'EPHYS_1_images_G_3uL_reward'
# g_sessions = behavior_sessions[mask]
# mask = (behavior_sessions['age_in_days'] > 150) & (behavior_sessions['age_in_days'] <= 170)

# g_sessions = g_sessions[mask]

# # behavior_sessions['session_type'].unique()
# display(g_sessions[['sex','genotype']].value_counts())

In [10]:
# display(g_sessions[g_sessions['sex']=='F'])
# # g_sessions['mouse_id'].value_counts()

In [70]:
# ecephys_sessions_list = [1056495334, 1061238668, 1124507277]
# name_list = ["wt", "vip", "sst"]	
# common_values = behavior_sessions['ecephys_session_id'].isin(ecephys_sessions_list)
# behavior_sessions[common_values]

In [ ]:
# ecephys_sessions_list = [1056495334, 1061238668, 1124507277]
ecephys_sessions_list = [1056495334]
# name_list = ["wt", "vip", "sst"]	
name_list = ["wt"]
common_values

In [12]:
for session_id, name in zip(ecephys_sessions_list, name_list):
    print("Loading session",session_id, name)
    globals()[f'session_{name}'] = cache.get_ecephys_session(ecephys_session_id=session_id)


Loading session 1056495334 wt


c:\myprograms\Anaconda\envs\behavis\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [34]:
session_wt.spike_times

{1061844816: array([  37.07814795,  793.88146488,  843.98570596, ..., 9591.98282433,
        9591.99355767, 9592.04505768]),
 1061844625: array([5.20057635e+00, 5.94880980e+00, 9.12661029e+00, ...,
        9.59081849e+03, 9.60489179e+03, 9.60647689e+03]),
 1061844616: array([4.47430958e+00, 4.57694292e+00, 4.75900962e+00, ...,
        9.59318259e+03, 9.59324819e+03, 9.59330739e+03]),
 1061844493: array([4.59924293e+00, 5.04287633e+00, 6.83690994e+00, ...,
        9.59025342e+03, 9.59210576e+03, 9.59218506e+03]),
 1061844492: array([  11.48301066,   13.35857761,   26.22861294, ..., 9591.89532432,
        9592.58905776, 9592.60312443]),
 1061844652: array([5.52787640e+00, 5.77327644e+00, 6.06810982e+00, ...,
        9.59181282e+03, 9.59196202e+03, 9.59880796e+03]),
 1061844651: array([  20.91444545,   34.80131426,   58.93471799, ..., 9589.43102394,
        9590.8857575 , 9591.7355243 ]),
 1061844467: array([4.31524288e+00, 4.86874297e+00, 4.97814299e+00, ...,
        9.59332282e+03, 9.59

# Slicing data by stimulus timestamps

In [30]:
stimulus_presentations = session_wt.stimulus_presentations
# stimulus_presentations.columns

In [14]:
mask = (stimulus_presentations['stimulus_block'] == 0) 
nat_imgs = stimulus_presentations[mask]
nat_imgs["omitted"].value_counts()    

False    4621
True      182
Name: omitted, dtype: Int64

In [16]:
mask = nat_imgs["omitted"]==True
idx = nat_imgs[mask].index.to_numpy()

# nat_imgs.iloc[all_idxs].head(20)

In [17]:
all_idxs = []
for i in idx:
    omitted_stim = nat_imgs.iloc[i-1]['image_name']
    all_idxs.append(i)
    all_idxs.append(i+1)
    
    count = 1
    stop = 0
    while stop == 0:
        if nat_imgs.iloc[i-count]['image_name'] == omitted_stim:
            all_idxs.append(i-count)
        else:
            stop = 1           
        count += 1   
all_idxs = list(set(all_idxs))
# len(all_idxs)  
all_idxs.sort()
omission_sequences = nat_imgs.iloc[all_idxs]    
    # all_idxs.append(i+1)

1847

In [27]:
mask = probes['ecephys_session_id']==1056495334
ecephys_session_probes = probes[mask]
probes_list = []
for row in ecephys_session_probes.iterrows():
    if not any(acronym in region_dict['midbrain'] for acronym in row[1]['structure_acronyms']):
        probes_list.append(row[0])
        print("Appended", row[0],row[1]['structure_acronyms'])
    
    # probes_list.append(row[1]['id'])
# if ecephys_session_probes['structure_acronyms']


Appended 1056630213 ['VISam', 'grey', 'root']
Appended 1056630214 ['VISpm', 'grey', 'root']
Appended 1056630215 ['VISp', 'grey', 'root']
Appended 1056630217 ['VISl', 'grey', 'root']
Appended 1056630218 ['VISal', 'grey', 'root']
Appended 1056630219 ['VISrl', 'grey', 'root']


In [32]:
dir(session_wt)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_behavior_session_id',
 '_date_of_acquisition',
 '_exclude_from_equals',
 '_eye_tracking',
 '_eye_tracking_rig_geometry',
 '_get_identifier',
 '_get_keywords',
 '_get_metadata',
 '_get_monitor_delay',
 '_get_probe',
 '_get_properties',
 '_get_session_type',
 '_is_value_self',
 '_licks',
 '_metadata',
 '_name',
 '_optotagging_table',
 '_probes',
 '_raw_running_speed',
 '_read_behavior_stimulus_timestamps',
 '_read_data_from_stimulus_file',
 '_read_eye_tracking_table',
 '_read_licks',
 '_read_rewards',
 '_read_session_timestamps',
 '_read_stimuli',
 '_read_trials',
 '_rew

In [31]:
session_wt.get_lfp(probes_list[0])


probe_probeA_lfp.nwb:  18%|█▊        | 852M/4.74G [20:42<1:39:01, 655kMB/s]    

In [ ]:
def align_lfp(lfp, trial_window, alignment_times, trial_ids = None):
    '''
    Aligns the LFP data array to experiment times of interest
    INPUTS:
        lfp: data array containing LFP data for one probe insertion
        trial_window: vector specifying the time points to excise around each alignment time
        alignment_times: experiment times around which to excise data
        trial_ids: indices in the session stim table specifying which stimuli to use for alignment.
                    None if aligning to non-stimulus times
    
    OUTPUT:
        aligned data array with dimensions channels x trials x time
    '''
    
    time_selection = np.concatenate([trial_window + t for t in alignment_times])
    
    if trial_ids is None:
        trial_ids = np.arange(len(alignment_times))
        
    inds = pd.MultiIndex.from_product((trial_ids, trial_window), 
                                      names=('presentation_id', 'time_from_presentation_onset'))

    ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
    ds = ds.assign(time=inds).unstack('time')

    return ds['aligned_lfp']

In [ ]:
presentation_times = omission_sequences['start_time'].values
presentation_ids = omission_sequences.index.values

In [ ]:
# below is aligned lfp data downsampled to 500 Hz
aligned_lfp = align_lfp(lfp, np.arange(-0.5, 0.5, 1/500), presentation_times, presentation_ids)

`aligned_lfp` is a DataArray with dimensions of channels x trials x time. It's been downsampled to 500 Hz by changing the time step in the `trial_window` argument of the `align_lfp` function.

Note that we can get the channels IDs for each channel in this DataArray. Let's use the session channels table to map these to the probe and mark the surface of the brain.

In [ ]:
chans = session.get_channels()
lfp_chan_depths = [chans.loc[c]['probe_vertical_position'] for c in lfp.channel.values]

chans_in_brain = chans[(chans['probe_id']==1064735073)&(~chans['structure_acronym'].str.contains('root'))]
first_channel_in_brain_position = chans_in_brain['probe_vertical_position'].max()